In [3]:
api_key='RGAPI-d8998b80-6be2-4d46-a31a-e7669f949e5d'
import requests

In [4]:
import pandas as pd

root= 'https://euw1.api.riotgames.com/'
challenger_root = 'lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
grandmaster_root = 'lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5'
master_root = 'lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5'

challenger_response = requests.get(root+challenger_root+'?api_key='+api_key)
grandmaster_response = requests.get(root+grandmaster_root+'?api_key='+api_key)
master_response = requests.get(root+master_root+'?api_key='+api_key)

challenger_df = pd.DataFrame(challenger_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
grandmaster_df=pd.DataFrame(grandmaster_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
master_df=pd.DataFrame(master_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)


#concatenating the dataframes
high_rank_ladder=pd.concat([challenger_df, grandmaster_df, master_df])[0:100].reset_index(drop=True)
high_rank_ladder=high_rank_ladder.drop(columns='rank').reset_index(drop=False).rename(columns={'index':'rank'})
high_rank_ladder['rank']+=1

high_rank_ladder

,rank,summonerId,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,1,5NzokD2VZC4DB3qBJdl6LOO3PiRQcORDRHZRLgSKdBtt0YA,1643,227,138,True,False,False,False
1,2,U4_ndlbviSSK2g20hBy5NNOG1awHFhM2iV_u4NlV73ayAd...,1513,435,342,True,False,False,False
2,3,jOdw_LVxNtdTh65T_VL7qRzrguVEKKkqQt9CUyL0f7fMdDI,1508,321,237,True,False,False,True
3,4,JGhpg_EeM7jdF-U90nyputqFS-hC3ftrDN---vtbuvo7uDzA,1504,235,159,True,False,False,False
4,5,KFCNaCBCxDEfbGcW3ltdr_of4XStG8Q1CLDY6OOFTcm1Kz0,1469,257,173,True,False,False,False
...,...,...,...,...,...,...,...,...,...
95,96,mdXd6ORXy6HjQmedLaUPRgIg0oihu4udlNc1CqSOkF9Eghk,1006,351,287,False,False,False,False
96,97,6I-wuss2jYMwDrKUrAdZKc_j2sY0jiYyE58Uhz3z1G3vdd...,1006,253,193,False,False,False,False
97,98,ysYyH3d8JeUtTkQE_jpTZXVcp_a9HATRLyO0lD_bS3AT_N0,1004,124,62,False,False,True,True
98,99,3kMtK73NvZuLjD6hrCJB3cEGB_oFXJhtlz_DnNeKinSxLw...,1003,168,118,False,False,False,False


In [5]:
import time
def get_puuid_from_summoner_id(summonerId=None):
    link=f'https://euw1.api.riotgames.com/lol/summoner/v4/summoners/{summonerId}?api_key={api_key}'

    while True:
        response = requests.get(link)
        if response.status_code==429:
            print('rate limit exceeded')
            time.sleep(10)
            continue
        puuid=response.json()['puuid']
        return puuid

In [6]:
puuid_list = []

for summoner in high_rank_ladder['summonerId']:
    puuids = get_puuid_from_summoner_id(summonerId=summoner)
    puuid_list.append(puuids)

rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded
rate limit exceeded


In [7]:
puuid_list[0:5]

['buvMukA-JVre-d3tH8WGdH0Fewk5VwFKmuVog33gO7PDRcf_-BV4Iy6jCV0DsaN_HfPU1Or2suJVSA',
 'rH_mM7PpbVDWYWZeAY9_wh_CKd4_lTUbS8dMpMRxfOvZDNp_wQVAG_KGOORmnlDhViUhIyNIuES0hQ',
 'aZFgbLH6q7UTdHCTLmcBIvarGyRb38eEzW0b_jwnmFx90Ur988DDZvv9kSyoqOedO_rIM5b6Y2VEgg',
 'A4-eBTYpPDIpHuBu2ChYaB7Jq2Ja5YXAW5LBwBjVjljKnmv3kS98yPQXotPL6tURenUmBhUJplH4mA',
 'GFaoHL-BXl_WQ7ni6OoBY4PI8vIaPFsLUo5EH0WN5QNX-j63OFSF79byTEhwHsxqsl31Ifpt7AcqwQ']

In [8]:
def get_match_id_from_puuid(puuid=None):
    """
    Function to retrieve the most recent 20 match id's. Takes a player's puuid as argument and outputs the match id's of their most recent 20 ranked matches.
    Queue and type must remain as 420 and ranked respectively as this will only get the ranked matches which is what we want.
    """
    link=f'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?queue=420&type=ranked&start=0&count=20&api_key={api_key}'
    while True:
        response = requests.get(link)
        if response.status_code==429:
            print('rate limit exceeded')
            time.sleep(10)
            continue
        match_id_list=response.json()
        break
    return match_id_list


In [9]:
all_match_id_list=[]

for player_puuid in puuid_list:
    match_ids=get_match_id_from_puuid(puuid=player_puuid)
    all_match_id_list.extend(match_ids)

In [10]:
all_match_id_list[0:5]

['EUW1_6928661440',
 'EUW1_6928550011',
 'EUW1_6928471250',
 'EUW1_6927403998',
 'EUW1_6926350115']

In [11]:
len(all_match_id_list)

2000

# Removing duplicates from match lists (they are the same matches)

In [12]:
from collections import Counter

# Count the occurrences of each match ID
match_id_counts = Counter(all_match_id_list)

# Find the duplicate match IDs
duplicate_match_ids = [match_id for match_id, count in match_id_counts.items() if count > 1]

# Remove duplicate match IDs from the list
unique_match_id_list = list(set(all_match_id_list))

print(f"Original number of match IDs: {len(all_match_id_list)}")
print(f"Number of duplicate match IDs: {len(duplicate_match_ids)}")
print(f"Number of unique match IDs after removal: {len(unique_match_id_list)}")


Original number of match IDs: 2000
Number of duplicate match IDs: 393
Number of unique match IDs after removal: 1419


In [13]:
unique_match_id_list[0:5]

['EUW1_6926030077',
 'EUW1_6925891806',
 'EUW1_6912141732',
 'EUW1_6905212859',
 'EUW1_6922441165']

# Retrieivng entire match JSON given a match ID

In [14]:
def get_match_info_from_id(matchId=None):
    link= f'https://europe.api.riotgames.com/lol/match/v5/matches/{matchId}?api_key={api_key}'
    while True:
        response = requests.get(link)
        if response.status_code==429:
            print('rate limit exceeded')
            time.sleep(10)
            continue
        elif response.status_code==200:
            break
        elif response.status_code==404:
            return 0
        else:
            print('Response is bad: '+str(response.status_code))
    match_data=response.json()
    return match_data

# Retrieving specific match information that I am interested in

In [41]:
import pandas as pd

def get_specific_match_info(data):
    info = data['info']
    metadata = data['metadata']
    participants = metadata['participants']  # List of player puuids
    teams = info['teams']
    
    #specific stats that applies to every player in the match
    game_duration = info['gameDuration']
    game_ended_in_surrender = info['participants'][0]['gameEndedInSurrender']
    game_ended_in_early_surrender=info['participants'][0]['gameEndedInEarlySurrender']
    blue_side_won = info['participants'][0]['win']
    

    #objectives will be considered separately because its in teamsDTO and not participantDTO

    #blue side objectives
    blue_side_tower = teams[0]['objectives']['tower']['kills']
    blue_side_dragon = teams[0]['objectives']['dragon']['kills']
    blue_side_baron = teams[0]['objectives']['baron']['kills']
    blue_side_riftHerald = teams[0]['objectives']['riftHerald']['kills']
    blue_side_horde = teams[0]['objectives']['horde']['kills']
    blue_side_inhibitor = teams[0]['objectives']['inhibitor']['kills']

    #red side objectives
    red_side_tower = teams[1]['objectives']['tower']['kills']
    red_side_dragon = teams[1]['objectives']['dragon']['kills']
    red_side_baron =teams[1]['objectives']['baron']['kills']
    red_side_riftHerald =teams[1]['objectives']['riftHerald']['kills']
    red_side_horde = teams[1]['objectives']['horde']['kills']
    red_side_inhibitor = teams[1]['objectives']['inhibitor']['kills']


    #build dictionaries for both blue adn red sides stats
    blue_side_stats = {
        'minions_killed': 0,
        'kills': 0,
        'deaths': 0,
        'assists': 0,
        'gold_earned': 0,
        'gold_spent': 0,
        'wards_killed':0,
        'wards_placed':0,
        'vision_score':0
    }

    red_side_stats = {
        'minions_killed': 0,
        'kills': 0,
        'deaths': 0,
        'assists': 0,
        'gold_earned': 0,
        'gold_spent': 0,
        'wards_killed':0,
        'wards_placed':0,
        'vision_score':0
    }
    
    
    #iterate through each players data. if the index of their participant id/puuid is less than 5 then they are on the blue team, otherwise they are on red team
    #side_stats is therefore assigned to each side depending on whether the participant/player id (i) is less than or greater than 5.
    for i, player in enumerate(info['participants']):
        if i < 5:
            side_stats=blue_side_stats
        else:
            side_stats=red_side_stats

        neautral_minions_killed = player['neutralMinionsKilled']
        minions_killed = player['totalMinionsKilled']
        kills = player['kills']
        assists = player['assists']
        deaths = player['deaths']
        gold_earned = player['goldEarned']
        gold_spent = player['goldSpent']
        wards_killed = player['wardsKilled']
        wards_placed=player['wardsPlaced']
        vision_score=player['visionScore']

        #add up the stats for both blue and red sides
        side_stats['minions_killed'] += neautral_minions_killed+minions_killed
        side_stats['assists'] += assists
        side_stats['deaths'] += deaths
        side_stats['kills'] += kills
        side_stats['gold_earned']+=gold_earned
        side_stats['gold_spent']+=gold_spent
        side_stats['wards_killed']+=wards_killed
        side_stats['wards_placed']+=wards_placed
        side_stats['vision_score']+=vision_score

    #create a dataframe with blue and red side stats
    df = pd.DataFrame({
        #game info
        'game_duration':[game_duration],
        'game_ended_in_surrender': [game_ended_in_surrender],
        'game_ended_in_early_surrender': [game_ended_in_early_surrender],
        'blue_side_won': [blue_side_won],

        #blue side
        'total_blue_side_minions_killed': [blue_side_stats['minions_killed']],
        'total_blue_side_kills': [blue_side_stats['kills']],
        'total_blue_side_deaths': [blue_side_stats['deaths']],
        'total_blue_side_assists': [blue_side_stats['assists']],
        'total_blue_side_gold_earned':[blue_side_stats['gold_earned']],
        'total_blue_side_gold_spent':[blue_side_stats['gold_spent']],
        'total_blue_side_wards_killed':[blue_side_stats['wards_killed']],
        'total_blue_side_wards_placed':[blue_side_stats['wards_placed']],
        'total_blue_side_vision_score':[blue_side_stats['vision_score']],
        'total_blue_side_towers_taken': [blue_side_tower],
        'total_blue_side_inhibitors_destroyed': [blue_side_inhibitor],
        'total_blue_side_dragons_killed': [blue_side_dragon],
        'total_blue_side_horde': [blue_side_horde],
        'total_blue_side_baron':[blue_side_baron],
        'total_blue_side_riftHerald':[blue_side_riftHerald],

        #red side
        'total_red_side_minions_killed': [red_side_stats['minions_killed']],
        'total_red_side_kills': [red_side_stats['kills']],
        'total_red_side_deaths': [red_side_stats['deaths']],
        'total_red_side_assists': [red_side_stats['assists']],
        'total_red_side_gold_earned':[red_side_stats['gold_earned']],
        'total_red_side_gold_spent':[red_side_stats['gold_spent']],
        'total_red_side_ward_takedowns':[red_side_stats['wards_killed']],
        'total_red_side_wards_placed':[red_side_stats['wards_placed']],
        'total_red_side_vision_score':[red_side_stats['vision_score']],
        'total_red_side_towers_taken': [red_side_tower],
        'total_red_side_inhibitors_destroyed': [red_side_inhibitor],
        'total_red_side_dragons_killed': [red_side_dragon],
        'total_red_side_horde': [red_side_horde],
        'total_red_side_baron':[red_side_baron],
        'total_red_side_riftHerald':[red_side_riftHerald]

    })

    return df



In [42]:
get_specific_match_info(data=get_match_info_from_id(matchId='EUW1_6909754606'))

,game_duration,game_ended_in_surrender,game_ended_in_early_surrender,blue_side_won,total_blue_side_minions_killed,total_blue_side_kills,total_blue_side_deaths,total_blue_side_assists,total_blue_side_gold_earned,total_blue_side_gold_spent,...,total_red_side_gold_spent,total_red_side_ward_takedowns,total_red_side_wards_placed,total_red_side_vision_score,total_red_side_towers_taken,total_red_side_inhibitors_destroyed,total_red_side_dragons_killed,total_red_side_horde,total_red_side_baron,total_red_side_riftHerald
0,1521,True,False,True,832,34,22,53,59039,53325,...,50350,38,70,180,3,0,3,0,0,1


In [43]:
#get_match_info_from_id(matchId='EUW1_6909754606')

In [39]:
def test_function(match_data):

    info=match_data['info']
    teams=info['teams']


    game_creation = info['gameCreation']
    game_duration = info['gameDuration']


    '''for team in teams:
        objectives = team['objectives']
        baron_nash = objectives['baron']
        dragons=objectives['dragon']
        void_grubs = objectives['horde']
        rift_herald = objectives['riftHerald']
        tower = objectives['tower']
        inhibitor=objectives['inhibitor']

        #for objective in [baron_nash, dragons,void_grubs,rift_herald,tower,inhibitor]:
            #first=objective['first']
            #objective_kills=objective['kills']'''


    return teams[1]['objectives']

In [44]:
#test_function(match_data=get_match_info_from_id(matchId='EUW1_6909754606'))

In [51]:
len(unique_match_id_list)

1419

In [52]:
match_ids = unique_match_id_list[0:20]

df=pd.DataFrame()

for match_id in match_ids:
    game = get_match_info_from_id(matchId=match_id)
    matchdf=get_specific_match_info(game)

    df= pd.concat([df,matchdf])

In [53]:
df

,game_duration,game_ended_in_surrender,game_ended_in_early_surrender,blue_side_won,total_blue_side_minions_killed,total_blue_side_kills,total_blue_side_deaths,total_blue_side_assists,total_blue_side_gold_earned,total_blue_side_gold_spent,...,total_red_side_gold_spent,total_red_side_ward_takedowns,total_red_side_wards_placed,total_red_side_vision_score,total_red_side_towers_taken,total_red_side_inhibitors_destroyed,total_red_side_dragons_killed,total_red_side_horde,total_red_side_baron,total_red_side_riftHerald
0,1241,True,False,True,631,30,11,46,46043,40400,...,33775,14,38,100,2,0,2,0,0,0
0,1315,False,False,False,644,11,31,14,37085,35750,...,46350,14,57,131,10,1,3,6,1,1
0,1773,False,False,False,793,21,24,46,55303,53775,...,52050,27,75,161,8,1,1,0,1,0
0,2045,False,False,False,997,39,41,57,72410,66660,...,66100,32,87,195,9,2,2,4,0,0
0,1371,True,False,False,687,11,23,13,38993,37375,...,43490,28,38,138,5,0,3,0,1,0
0,1320,False,False,False,596,7,27,10,34975,30900,...,42935,18,43,114,10,1,2,5,1,1
0,2125,False,False,False,1099,35,36,68,72452,67050,...,70975,54,81,264,11,3,3,5,1,1
0,1335,False,False,True,678,31,11,47,53043,46125,...,34200,8,46,91,1,0,1,0,0,0
0,926,True,False,False,447,13,23,14,29268,26645,...,28525,9,42,90,3,0,1,3,0,1
0,1663,False,False,False,699,24,35,53,51102,46385,...,56713,27,56,160,11,2,2,2,0,1


In [ ]:
unique_match_id_list[]

In [47]:
df.to_csv('test.csv')

In [50]:
#get_match_info_from_id(matchId='EUW1_6909754606')

In [48]:
df2=pd.read_csv('test.csv')


In [54]:
df2.columns

Index(['Unnamed: 0', 'game_duration', 'game_ended_in_surrender',
       'game_ended_in_early_surrender', 'blue_side_won',
       'total_blue_side_minions_killed', 'total_blue_side_kills',
       'total_blue_side_deaths', 'total_blue_side_assists',
       'total_blue_side_gold_earned', 'total_blue_side_gold_spent',
       'total_blue_side_wards_killed', 'total_blue_side_wards_placed',
       'total_blue_side_vision_score', 'total_blue_side_towers_taken',
       'total_blue_side_inhibitors_destroyed',
       'total_blue_side_dragons_killed', 'total_blue_side_horde',
       'total_blue_side_baron', 'total_blue_side_riftHerald',
       'total_red_side_minions_killed', 'total_red_side_kills',
       'total_red_side_deaths', 'total_red_side_assists',
       'total_red_side_gold_earned', 'total_red_side_gold_spent',
       'total_red_side_ward_takedowns', 'total_red_side_wards_placed',
       'total_red_side_vision_score', 'total_red_side_towers_taken',
       'total_red_side_inhibitors_destr